In [2]:
import pandas as pd
import csv
import os
import seaborn as sns

from sklearn.metrics import f1_score

In [3]:
# load csv files with results from different models
results = {}
directory = '../0_results/month-models'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        results[os.path.splitext(filename)[0]] = pd.read_csv(os.path.join(directory, filename))
        continue
    else:
        continue

In [4]:
# calculate f1 
f1_type = 'macro'
scores = {}

for key in results:
    scores[key] = f1_score(results[key]['label'], results[key]['prediction'], average=f1_type)

score_series = {}
for model in sorted(pd.unique(pd.Series(scores.keys()).str[:17])):
    score_series[model] = []
    for key in sorted(scores):
        if model in key:
            score_series[model].append(scores[key])

out_df = pd.DataFrame.from_dict(score_series).reset_index().rename(columns={'index':'y-m'})
out_df.columns = out_df.columns.str.replace('bert-base-', '')
out_df['y-m'] = out_df['y-m'].apply(lambda x: f'2018-0{x+1}')

out_df['mean'] = out_df.mean(axis=1)

def color(val):
    color = 'red' if val < 0.9 else 'green'
    return 'background-color: %s' % color

cm = sns.light_palette('green', as_cmap=True)
out_df.style.background_gradient(cmap=cm, low = 0, high = 1.0)

,y-m,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,mean
0,2018-01,0.940387,0.736661,0.715535,0.744079,0.702428,0.734025,0.678463,0.717885,0.699874,0.748956,0.741829
1,2018-02,0.689663,0.950048,0.694316,0.712014,0.698271,0.717885,0.696911,0.704574,0.679659,0.745861,0.728920
2,2018-03,0.689814,0.698234,0.971772,0.698234,0.672320,0.686763,0.657073,0.665956,0.674123,0.712934,0.712722
3,2018-04,0.675328,0.693497,0.670072,0.975497,0.666316,0.664313,0.643296,0.669614,0.663499,0.706946,0.702838
4,2018-05,0.682241,0.696841,0.690471,0.717119,0.955167,0.677128,0.687987,0.698140,0.676617,0.735794,0.721751
5,2018-06,0.676206,0.693069,0.675176,0.701119,0.648140,0.950862,0.661255,0.690703,0.655733,0.680440,0.703270
6,2018-07,0.706239,0.691209,0.686672,0.706404,0.696527,0.684368,0.974955,0.676510,0.685712,0.713108,0.722170
7,2018-08,0.671950,0.696212,0.674759,0.719539,0.699397,0.708428,0.681160,0.941435,0.661653,0.720173,0.717471
8,2018-09,0.700295,0.700951,0.702004,0.682937,0.706371,0.676680,0.668327,0.672725,0.964341,0.718709,0.719334
9,2018-010,0.702875,0.708436,0.718805,0.718556,0.691792,0.673970,0.670800,0.702523,0.707446,0.967241,0.726245
